In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
print(np.__version__)
import numba
print(numba.__version__)
import umap
import networkx as nx
from math import ceil, sqrt
from matplotlib.colors import Normalize
import matplotlib.cm as cm

# PCA Data Visualization

Setting up entire dataset

In [ ]:
PCA = pd.read_csv("PCAoutputs/pca_results.eigenvec", sep='\s+', header=0)
pop_data = pd.read_csv('data/20130606_g1k_3202_samples_ped_population.txt', sep='\s+')

# display(PCA.columns)
# display(pop_data[['FamilyID', 'SampleID']].iloc[44])
# display(PCA[['#IID']].iloc[44])
# display(pop_data.head)
# display(PCA.head)

# merge
merged = PCA.merge(pop_data, left_on='#IID', right_on='SampleID', how='left')
merged.columns = merged.columns.str.strip()
display(merged.head())



Creating a heatmap of all PCs against population, and cell represents mean PC score

In [ ]:
# making PCA plots
## Heatmap of all PCA:

# Finding the mean PC score for each pop
mean_pcs = merged.groupby('Population')[[f'PC{i+1}' for i in range(20)]].mean()
global_std_pcs = merged[[f'PC{i+1}' for i in range(20)]].std()
mean_standerdized_pcsByPop = mean_pcs / global_std_pcs
display(mean_standerdized_pcsByPop.head())

# Heatmap
fig1, (ax3) = plt.subplots(figsize=(9, 6))
heatmap = sns.heatmap(
    mean_standerdized_pcsByPop,
    cmap='vlag',            
    center=0, # 0 = global average
    annot=False,            
    xticklabels=True,
    yticklabels=True,
    ax = ax3,
    cbar_kws={'label': 'Mean PC Score'}
)
ax3.set_title("Standardized Mean PCs by Pop")
ax3.set_xlabel("PCs")
ax3.set_ylabel("Population")
ax3.set_xticklabels(ax3.get_xticklabels(), fontsize=8)
ax3.set_yticklabels(ax3.get_yticklabels(), fontsize=8)
plt.tight_layout()
plt.show()
plt.close(fig1)

Scatter plot of PC1 against PC2 colored by superpop. Then Umap colored by superpop.

In [ ]:

fig2, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 9))
# Normal Scatter
sns.scatterplot( # ax1
    data=merged,
    x='PC1',
    y='PC2',
    hue='Superpopulation',
    palette='tab10',
    ax=ax1,
    s=10
)
ax1.set_title("PCA: PC1 vs PC2 colored by Superpopulation")
ax1.set_xlabel("PC1")
ax1.set_ylabel("PC2")
ax1.legend(title='Superpopulation', bbox_to_anchor=(1, 1), loc='upper right', ncol=1)


# Umap
pcs_only = merged[[f'PC{i+1}' for i in range(20)]]

# Fit and transform with UMAP
reducer = umap.UMAP(random_state=100)
embedding = reducer.fit_transform(pcs_only)

# Create a DataFrame for plotting
umap_df = merged.copy()
umap_df['UMAP1'] = embedding[:, 0]
umap_df['UMAP2'] = embedding[:, 1]

# Plotting the umap
sns.scatterplot( #ax2
    data=umap_df,
    x='UMAP1',
    y='UMAP2',
    hue='Superpopulation',
    palette='tab10',
    s=10,
    ax = ax2
)
ax2.set_title("UMAP of Individuals Colored by Population")
ax2.set_xlabel('UMAP1')
ax2.set_ylabel('UMAP2')

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.show()
plt.close(fig2)

# Kinship

Setting up data

In [ ]:
kinship = pd.read_csv("kin_outputs/king_results.kin0", sep='\s+', header=0)
kinship_filtered = kinship[kinship['KINSHIP'] >= 0.0884].copy().reset_index(drop=True)
kinship_filtered['KINSHIP_INF'] = np.log1p(kinship_filtered['KINSHIP'])
display(kinship_filtered.head())
print(kinship_filtered.shape[0]) 

Making a network graph

In [ ]:
# Build graph
G = nx.Graph()
for _, r in kinship_filtered.iterrows():
    G.add_edge(r['IID1'], r['IID2'], w=float(r['KINSHIP']))

if G.number_of_edges() == 0:
    raise ValueError("No edges after filtering.")

# Components
comps = sorted((G.subgraph(c).copy() for c in nx.connected_components(G)),
               key=lambda g: g.number_of_nodes(), reverse=True)

# Layout grid
K = min(12, len(comps))
rows = int(ceil(sqrt(K)))
cols = int(ceil(K / rows))
fig, axes = plt.subplots(rows, cols, figsize=(3.5*cols, 3.5*rows))
axes = np.ravel(axes)

# Color scale
vmin, vmax = 0.0884, max(0.35, kinship_filtered['KINSHIP'].max())
norm = Normalize(vmin=vmin, vmax=vmax)
cmap = cm.viridis

for i, H in enumerate(comps[:K]):
    ax = axes[i]
    pos = nx.spring_layout(H, weight='w', seed=0, k=1.4)  # k↑ = more spacing
    phi = [H.edges[e]['w'] for e in H.edges()]
    edge_colors = [cmap(norm(p)) for p in phi]

    nx.draw_networkx_edges(H, pos, ax=ax, edge_color=edge_colors, width=1.2, alpha=0.9)
    nx.draw_networkx_nodes(H, pos, ax=ax, node_size=60)  # smaller nodes
    nx.draw_networkx_labels(H, pos, font_size=6, ax=ax)  # all nodes labeled
    ax.set_title(f"Comp {i+1} (n={H.number_of_nodes()})", fontsize=8)
    ax.axis("off")

# Hide unused axes
for j in range(i+1, len(axes)):
    axes[j].axis("off")

# Dedicated colorbar axis
cbar_ax = fig.add_axes([0.92, 0.15, 0.015, 0.7])  # [left, bottom, width, height]
sm = cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax, label="Kinship coefficient φ")

plt.subplots_adjust(right=0.9)  # make space for cbar
plt.show()

# GWAS

There exists no phenotype/trait within 1000genome project. This renders a meaningful GWAS impossible. However, we can arbitrarly assign 1s and 0s to the data and perform a GWAS on this "phenotype", however it will ofcourse be uninfromative and insubstantial. Nonetheless, GWAS architecture remains the same.

In [ ]:
#!/usr/bin/env python3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

in_path = "GWAS_outputs/gwas.PHENO1.glm.logistic"

df = pd.read_csv(in_path, delim_whitespace=True, comment="#")
if 'CHROM' not in df.columns and '#CHROM' in df.columns:
    df = df.rename(columns={'#CHROM':'CHROM'})

if 'TEST' in df.columns:
    df = df[df['TEST'].eq('ADD')].copy()

for col in ['CHROM','POS','P']:
    if col not in df.columns:
        raise ValueError(f"required column '{col}' not found in {in_path}")

chrom_map = {
    'X': 23, 'Y': 24, 'MT': 25, 'M': 25, 'chrX': 23, 'chrY': 24, 'chrM': 25, 'chrMT': 25
}

def to_chr_numeric(x):
    s = str(x)
    s = s.replace('chr','')
    if s in chrom_map:
        return chrom_map[s]
    try:
        return int(s)
    except ValueError:
        return np.nan

df['CHR'] = df['CHROM'].apply(to_chr_numeric)
df = df.dropna(subset=['CHR','POS','P']).copy()
df['CHR'] = df['CHR'].astype(int)

chrom_order = sorted(df['CHR'].unique())
chrom_sizes = df.groupby('CHR')['POS'].max().sort_index()
chrom_cumstart = {}
running = 0
for c in chrom_order:
    chrom_cumstart[c] = running
    running += chrom_sizes.get(c, 0)

df['BPcum'] = df.apply(lambda r: r['POS'] + chrom_cumstart[r['CHR']], axis=1)

df = df[df['P'] > 0].copy()
df['mlog10p'] = -np.log10(df['P'])

xticks = []
xlabels = []
for c in chrom_order:
    start = chrom_cumstart[c]
    end = start + chrom_sizes.get(c, 0)
    xticks.append((start + end) / 2)
    xlabels.append(str(c))

plt.figure(figsize=(12,5))
colors = ["C0", "C1"]
for i, c in enumerate(chrom_order):
    idx = df['CHR'].eq(c)
    plt.scatter(df.loc[idx, 'BPcum'], df.loc[idx, 'mlog10p'], s=4, alpha=0.7, c=colors[i % 2])

plt.axhline(-np.log10(5e-8), linestyle='--', linewidth=1)
plt.axhline(-np.log10(1e-5), linestyle=':', linewidth=1)

plt.xticks(xticks, xlabels, fontsize=8)
plt.xlabel('chromosome')
plt.ylabel('-log10(p)')
plt.title('Manhattan plot: gwas.PHENO1.glm.logistic (ADD)')
plt.margins(x=0)
plt.tight_layout()
plt.show()